In [1]:
# !python -m venv myenv

In [2]:
# !myenv\Scripts\activate.ps1

# !env_ml\Scripts\activate


In [1]:
import pandas as pd
from datasets import load_dataset

# # Loading the dataset from Hugging Face
dataset = load_dataset("SetFit/enron_spam")

# # Convert the training set to a Pandas DataFrame
df = pd.DataFrame(dataset['train'])
test=pd.DataFrame(dataset['test'])

C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.


In [2]:
train_data=df
train_data.head()


,message_id,text,label,label_text,subject,message,date
0,33214,any software just for 15 $ - 99 $ understandin...,1,spam,any software just for 15 $ - 99 $,understanding oem software\nlead me not into t...,2005-06-18
1,11929,perspective on ferc regulatory action client c...,0,ham,perspective on ferc regulatory action client c...,"19 th , 2 : 00 pm edt\nperspective on ferc reg...",2001-06-19
2,19784,wanted to try ci 4 lis but thought it was way ...,1,spam,wanted to try ci 4 lis but thought it was way ...,viagra at $ 1 . 12 per dose\nready to boost yo...,2004-09-11
3,2209,"enron / hpl actuals for december 11 , 2000 tec...",0,ham,"enron / hpl actuals for december 11 , 2000",teco tap 30 . 000 / enron ; 120 . 000 / hpl ga...,2000-12-12
4,15880,looking for cheap high - quality software ? ro...,1,spam,looking for cheap high - quality software ? ro...,"water past also , burn , course . gave country...",2005-02-13


In [3]:
test_data=test
test_data.head()

,message_id,text,label,label_text,subject,message,date
0,31329,expande tu imagen ! ! ! ! ! ! ! ! ! si no pued...,1,spam,expande tu imagen ! ! ! ! ! ! ! ! !,"si no puede ver este mail , entre a : http : /...",2005-01-19
1,3969,paliourg learning for life enlarge your member...,1,spam,paliourg learning for life,enlarge your member\nzenextend enlargement pil...,2004-05-06
2,27070,"cure premature ejaculation hello ,\ndid you ej...",1,spam,cure premature ejaculation,"hello ,\ndid you ejaculate before or within a ...",2005-07-17
3,2779,re : noms / actual flow for 3 / 19 / 01 we agr...,0,ham,re : noms / actual flow for 3 / 19 / 01,"we agree\n"" eileen ponton "" on 03 / 20 / 2001 ...",2001-03-20
4,2949,ehronline web address change this message is i...,0,ham,ehronline web address change,this message is intended for ehronline users o...,2001-03-30


In [4]:
df = pd.concat([train_data, test_data], ignore_index=True)
print(df.shape)
print(len(df))

(33716, 7)
33716


In [5]:
# # df
# Drop rows where 'text_joined' is empty
df = df[df['text'].str.len() > 0]

print(df[df['text'].str.len() == 0])  # Find any empty texts


# Optionally, reset the index if needed
df.reset_index(drop=True, inplace=True)

# Print the shape of the cleaned DataFrame to verify
print(f'Cleaned DataFrame shape: {df.shape}')


Empty DataFrame
Columns: [message_id, text, label, label_text, subject, message, date]
Index: []
Cleaned DataFrame shape: (33665, 7)


In [6]:
print(df.isnull().sum())


message_id    0
text          0
label         0
label_text    0
subject       0
message       0
date          0
dtype: int64


***

In [7]:
counts = df['label'].value_counts()
print(counts)

# Calculate percentages
percentages = (counts / len(df)) * 100

# Display the results
print(f"Percentage of ham (0): {percentages.get(0, 0):.2f}%")
print(f"Percentage of spam (1): {percentages.get(1, 0):.2f}%")

label
1    17120
0    16545
Name: count, dtype: int64
Percentage of ham (0): 49.15%
Percentage of spam (1): 50.85%


***

In [8]:
# ## Preprocessing
# ## Droping columns for train and test dataset.
# train_data= train_data.drop(columns=['message_id', 'label_text', 'date', 'subject','message'])
df =df.drop(columns=['message_id', 'label_text', 'date', 'message', 'subject'])
df.head()

,text,label
0,any software just for 15 $ - 99 $ understandin...,1
1,perspective on ferc regulatory action client c...,0
2,wanted to try ci 4 lis but thought it was way ...,1
3,"enron / hpl actuals for december 11 , 2000 tec...",0
4,looking for cheap high - quality software ? ro...,1


***

In [9]:
# ## Text Normalization
# ## Lowercasing
# # List of columns to apply lowercasing
columns_to_lower = ['text']  # Replace with your actual column names

# # Apply str.lower() to each column in the list
# # train_data[columns_to_lower] = train_data[columns_to_lower].apply(lambda x: x.str.lower())
df[columns_to_lower] = df[columns_to_lower].apply(lambda x: x.str.lower())
df.head()


,text,label
0,any software just for 15 $ - 99 $ understandin...,1
1,perspective on ferc regulatory action client c...,0
2,wanted to try ci 4 lis but thought it was way ...,1
3,"enron / hpl actuals for december 11 , 2000 tec...",0
4,looking for cheap high - quality software ? ro...,1


***

In [10]:
# ## Removing punctutaitons 
import string
df['text'] = df['text'].str.translate(str.maketrans('', '', string.punctuation))
df.shape

(33665, 2)

***

In [11]:
# # Removing numbers and special charectors
df['text'] = df['text'].str.replace(r'\d+', '', regex=True)
df

,text,label
0,any software just for understanding oem s...,1
1,perspective on ferc regulatory action client c...,0
2,wanted to try ci lis but thought it was way t...,1
3,enron hpl actuals for december teco tap ...,0
4,looking for cheap high quality software rota...,1
...,...,...
33660,re book notes vince look forward to meeting ...,0
33661,rollout schedule for unify real time deal pa...,0
33662,anshuman shrivastava sandeep vince has asked ...,0
33663,re good morning john \ni shall see christie t...,0


***

In [12]:
# #Removing WHite spaces
df['text'] = df['text'].str.strip()
df

,text,label
0,any software just for understanding oem s...,1
1,perspective on ferc regulatory action client c...,0
2,wanted to try ci lis but thought it was way t...,1
3,enron hpl actuals for december teco tap ...,0
4,looking for cheap high quality software rota...,1
...,...,...
33660,re book notes vince look forward to meeting ...,0
33661,rollout schedule for unify real time deal pa...,0
33662,anshuman shrivastava sandeep vince has asked ...,0
33663,re good morning john \ni shall see christie t...,0


***

In [13]:
# import nltk 
# nltk.download('punkt_tab')
# nltk.download('wordnet')
# nltk.download('stopwords')

In [14]:
## Tokenization 
from nltk.tokenize import word_tokenize
df['text_tokens'] = df['text'].apply(word_tokenize)
df.head()

,text,label,text_tokens
0,any software just for understanding oem s...,1,"[any, software, just, for, understanding, oem,..."
1,perspective on ferc regulatory action client c...,0,"[perspective, on, ferc, regulatory, action, cl..."
2,wanted to try ci lis but thought it was way t...,1,"[wanted, to, try, ci, lis, but, thought, it, w..."
3,enron hpl actuals for december teco tap ...,0,"[enron, hpl, actuals, for, december, teco, tap..."
4,looking for cheap high quality software rota...,1,"[looking, for, cheap, high, quality, software,..."


In [15]:
df.shape

(33665, 3)

***

In [16]:
## STOPWORDS REMOVAL 
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

df['text_tokens'] = df['text_tokens'].apply(lambda x: [word for word in x if word not in stop_words])
df.head()

,text,label,text_tokens
0,any software just for understanding oem s...,1,"[software, understanding, oem, software, lead,..."
1,perspective on ferc regulatory action client c...,0,"[perspective, ferc, regulatory, action, client..."
2,wanted to try ci lis but thought it was way t...,1,"[wanted, try, ci, lis, thought, way, expensive..."
3,enron hpl actuals for december teco tap ...,0,"[enron, hpl, actuals, december, teco, tap, enr..."
4,looking for cheap high quality software rota...,1,"[looking, cheap, high, quality, software, rota..."


In [17]:
df.shape

(33665, 3)

***

In [18]:
## Lemmatization 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

df['text_tokens'] = df['text_tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
df.head()

,text,label,text_tokens
0,any software just for understanding oem s...,1,"[software, understanding, oem, software, lead,..."
1,perspective on ferc regulatory action client c...,0,"[perspective, ferc, regulatory, action, client..."
2,wanted to try ci lis but thought it was way t...,1,"[wanted, try, ci, li, thought, way, expensive,..."
3,enron hpl actuals for december teco tap ...,0,"[enron, hpl, actuals, december, teco, tap, enr..."
4,looking for cheap high quality software rota...,1,"[looking, cheap, high, quality, software, rota..."


In [19]:
df.shape

(33665, 3)

***

In [20]:
## Joining words for tf-idf TRAIN
df['text_joined'] = df['text_tokens'].apply(lambda x: ' '.join(x))
df.head()

,text,label,text_tokens,text_joined
0,any software just for understanding oem s...,1,"[software, understanding, oem, software, lead,...",software understanding oem software lead tempt...
1,perspective on ferc regulatory action client c...,0,"[perspective, ferc, regulatory, action, client...",perspective ferc regulatory action client conf...
2,wanted to try ci lis but thought it was way t...,1,"[wanted, try, ci, li, thought, way, expensive,...",wanted try ci li thought way expensive viagra ...
3,enron hpl actuals for december teco tap ...,0,"[enron, hpl, actuals, december, teco, tap, enr...",enron hpl actuals december teco tap enron hpl ...
4,looking for cheap high quality software rota...,1,"[looking, cheap, high, quality, software, rota...",looking cheap high quality software rotated na...


In [21]:
df.shape

(33665, 4)

***

<h1>Vecotrization</h1>

<h2> Td-Idf </h2>

In [22]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

x = df["text_joined"]
y = df["label"]

# Step 1: Train-test split
X_train, X_test, y_train_tfidf, y_test_tfidf = train_test_split(x, y, test_size=0.2, random_state=42)

# Step 2: Initialize and fit the vectorizer on training data
tfidf = TfidfVectorizer(max_features=12000, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(X_train)

# Step 3: Transform the test data using the same vectorizer
X_test_tfidf = tfidf.transform(X_test)





In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import GridSearchCV, train_test_split,RandomizedSearchCV
# from sklearn.svm import SVC
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import make_scorer, accuracy_score, f1_score

# # Data Preparation
# x = df["text_joined"]  # Your text data
# y = df["label"]  # Labels

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# ### Set up the Pipeline ###
# # The pipeline will include the TF-IDF Vectorizer and the SVM model.
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer()),  # Step 1: TF-IDF Vectorizer
#     ('svm', SVC())                 # Step 2: SVM classifier
# ])

# # Define the parameter grid for both TF-IDF and SVM hyperparameters
# param_grid = {
#     'tfidf__max_features': [4000, 6000, 8000, 12000],           # TF-IDF parameters
#     'tfidf__ngram_range': [(1, 1), (1, 2)],
#     #'svm__C': [0.1, 1, 10],                        # SVM parameters
#     #'svm__kernel': ['linear', 'rbf']
#     'svm__cache_size':[10240] ,
#     'svm__max_iter':[1000]
# }

# # Set up GridSearchCV with accuracy as the scoring method
# # grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy',n_jobs=-1)
# random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_grid, n_iter=10, cv=5, random_state=42, n_jobs=-1)

# # Fit the grid search on the training data
# # grid_search.fit(X_train, y_train)
# random_search.fit(X_train, y_train)

# # Get the best parameters and estimator
# best_params = random_search.best_params_
# # best_estimator = grid_search.best_estimator_
# # print("Best parameters found by GridSearch:", best_params)
# print("Best Hyperparameters:", random_search.best_params_)

# # # Evaluate the best model on the test data
# # test_accuracy = best_estimator.score(X_test, y_test)
# # print(f"Test Accuracy: {test_accuracy:.4f}")
# # Evaluate on test data
# best_svc = random_search.best_estimator_
# accuracy = best_svc.score(X_test, y_test)
# print(f"Test Accuracy: {accuracy}")



In [24]:
print(X_train_tfidf.shape[0])
print(X_test_tfidf.shape[0])
print(X_train_tfidf.shape[0] + X_test_tfidf.shape[0])



26932
6733
33665


<h2> Word2Vec </h2>

In [25]:
### 2. Word2Vec Embeddings ###
import gensim
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Initialize the data
x= df['text_tokens']  
y = df['label']

X_train, X_test, y_train_w2v, y_test_w2v = train_test_split(x, y, test_size=0.2, random_state=42)


# Train the Word2Vec model on the training data
word2vec_model = Word2Vec(sentences=X_train, vector_size=300, window=7, min_count=3, workers=7, sg=1)

# Function to convert a sentence into its Word2Vec embeddings (average of word vectors)
def vectorize_sentence(sentence, model):
    words = [word for word in sentence if word in model.wv.key_to_index]  # Only words in vocab
    if len(words) == 0:
        return np.zeros(model.vector_size)  # Return zero vector if no words in the sentence
    return np.mean(model.wv[words], axis=0)

# Vectorize the training and testing data
X_train_w2v = np.array([vectorize_sentence(sentence, word2vec_model) for sentence in X_train])
X_test_w2v = np.array([vectorize_sentence(sentence, word2vec_model) for sentence in X_test])


In [26]:
# import pandas as pd
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.pipeline import Pipeline
# import gensim
# from gensim.models import Word2Vec
# import numpy as np
# from sklearn.metrics import accuracy_score
# from sklearn.base import BaseEstimator, TransformerMixin
# from scipy.stats import randint, uniform

# # Initialize the data
# x = df['text_tokens']  # Assume this column contains tokenized sentences (list of words per sentence)
# y = df['label']

# # Step 1: Train-test split
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# # Define a custom transformer for Word2Vec
# class Word2VecTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self, vector_size=100, window=5, min_count=1, sg=1, workers=6):
#         self.vector_size = vector_size
#         self.window = window
#         self.min_count = min_count
#         self.sg = sg  # Add 'sg' here to use CBOW or Skip-gram
#         self.workers = workers
#         self.model = None

#     def fit(self, X, y=None):
#         # Train Word2Vec model
#         self.model = Word2Vec(sentences=X, vector_size=self.vector_size, window=self.window, 
#                               min_count=self.min_count, sg=self.sg, workers=self.workers)
#         return self

#     def transform(self, X):
#         # Create sentence embeddings using the trained Word2Vec model
#         return np.array([self.vectorize_sentence(sentence) for sentence in X])

#     def vectorize_sentence(self, sentence):
#         words = [word for word in sentence if word in self.model.wv.key_to_index]
#         if len(words) == 0:
#             return np.zeros(self.model.vector_size)  # Return zero vector if no words in the sentence
#         return np.mean(self.model.wv[words], axis=0)

#     def get_params(self, deep=True):
#         # Required for RandomizedSearchCV
#         return {
#             'vector_size': self.vector_size,
#             'window': self.window,
#             'min_count': self.min_count,
#             'sg': self.sg,
#             'workers': self.workers
#         }

#     def set_params(self, **params):
#         # Required for RandomizedSearchCV
#         for key, value in params.items():
#             setattr(self, key, value)
#         return self

# # Step 2: Define the pipeline with Word2Vec and RandomForestClassifier
# pipeline = Pipeline([
#     ('word2vec', Word2VecTransformer()),  # Custom Word2Vec transformer
#     ('classifier', RandomForestClassifier(random_state=42))
# ])

# # Step 3: Define the parameter distributions for RandomizedSearchCV
# w2v_param_distributions = {
#     'word2vec__vector_size': randint(200, 400),  # Sample from a range of values
#     'word2vec__window': randint(5, 10),
#     'word2vec__min_count': randint(1, 5),
#     'word2vec__sg': [0, 1],  # CBOW or Skip-gram
#     'word2vec__workers': [8]  # Usually fixed based on hardware
# }

# # Step 4: Initialize RandomizedSearchCV with the pipeline
# random_search = RandomizedSearchCV(
#     pipeline, 
#     w2v_param_distributions, 
#     cv=5, 
#     scoring='accuracy',  # Specify the scoring metric here
#     n_jobs=-1,  # Use all available cores
#     n_iter=10,  # Number of random combinations to try
#     random_state=42  # For reproducibility
# )

# # Step 5: Fit RandomizedSearchCV on the training data
# random_search.fit(X_train, y_train)

# # Step 6: Access the best parameters and the best estimator
# print("Best Parameters:", random_search.best_params_)

# # Step 7: Optionally, evaluate the best classifier
# y_pred = random_search.predict(X_test)
# print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")


In [27]:
print(len(X_train_w2v))
print(len(X_test_w2v))
print(len(X_train_w2v) + len(X_test_w2v))
print(np.count_nonzero(X_train_w2v[0]))


26932
6733
33665
300


***

<h1>ALGORITHMS</h1>

<h2>1.SVM</h2>

<h2>OPTUNA</h2>

In [30]:
import optuna
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Assuming you have already done the train-test split and vectorization:
# X_train_tfidf, X_test_tfidf: TF-IDF vectorized train and test sets (sparse)
# X_train_w2v, X_test_w2v: Word2Vec vectorized train and test sets
# y_train, y_test: Corresponding labels

### Objective function for Optuna ###
def objective(trial, X_train, y_train, X_test, y_test):
    # Suggest hyperparameters for SVM
    C = trial.suggest_float('C', 0.1, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 4)
    else:
        degree = 3  # Default value for other kernels

    # Feature scaling for Word2Vec (dense matrix)
    if isinstance(X_train, np.ndarray):  
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    # Create and train SVM model
    svm = SVC(C=C, kernel=kernel, degree=degree, random_state=42,  cache_size=10240, max_iter=2000)
    svm.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = svm.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    return f1  # Optuna will maximize this metric

### Step 1: Optimize for TF-IDF ###
def optimize_tfidf():
    study_tfidf = optuna.create_study(direction='maximize')
    study_tfidf.optimize(lambda trial: objective(trial, X_train_tfidf, y_train_tfidf, X_test_tfidf, y_test_tfidf), n_trials=20)

    # Best hyperparameters from Optuna
    print("Best parameters for TF-IDF:", study_tfidf.best_params)
    print("Best F1 Score for TF-IDF:", study_tfidf.best_value)

    return study_tfidf.best_params

### Step 2: Optimize for Word2Vec ###
def optimize_w2v():
    study_w2v = optuna.create_study(direction='maximize')
    study_w2v.optimize(lambda trial: objective(trial, X_train_w2v, y_train_w2v, X_test_w2v, y_test_w2v), n_trials=20)

    # Best hyperparameters from Optuna
    print("Best parameters for Word2Vec:", study_w2v.best_params)
    print("Best F1 Score for Word2Vec:", study_w2v.best_value)

    return study_w2v.best_params

### Step 3: Run the optimization and evaluate ###
best_params_tfidf = optimize_tfidf()  # Optimize for TF-IDF
best_params_w2v = optimize_w2v()  # Optimize for Word2Vec

### Step 4: Train and evaluate the best model for TF-IDF ###
svm_tfidf_best = SVC(
    C=best_params_tfidf['C'],
    kernel=best_params_tfidf['kernel'],
    degree=best_params_tfidf.get('degree', 3),
    random_state=42,
    cache_size=10240,
    max_iter=1000
)

# Fit the model
svm_tfidf_best.fit(X_train_tfidf, y_train_tfidf)

# Predict and evaluate on the test set
y_pred_tfidf = svm_tfidf_best.predict(X_test_tfidf)
print(f"TF-IDF SVM Test Accuracy: {accuracy_score(y_test_tfidf, y_pred_tfidf):.4f}")
print(f"TF-IDF SVM Test Precision: {precision_score(y_test_tfidf, y_pred_tfidf):.4f}")
print(f"TF-IDF SVM Test Recall: {recall_score(y_test_tfidf, y_pred_tfidf):.4f}")
print(f"TF-IDF SVM Test F1 Score: {f1_score(y_test_tfidf, y_pred_tfidf):.4f}")
print(f"TF-IDF SVM Test Confusion Matrix:\n{confusion_matrix(y_test_tfidf, y_pred_tfidf)}")

### Step 5: Train and evaluate the best model for Word2Vec ###
svm_w2v_best = SVC(
    C=best_params_w2v['C'],
    kernel=best_params_w2v['kernel'],
    degree=best_params_w2v.get('degree', 3),
    random_state=42,
    cache_size=10240,
    max_iter=1000
)

# Standardize the Word2Vec data before fitting
scaler = StandardScaler()
X_train_w2v_scaled = scaler.fit_transform(X_train_w2v)
X_test_w2v_scaled = scaler.transform(X_test_w2v)

# Fit the model
svm_w2v_best.fit(X_train_w2v_scaled, y_train_w2v)

# Predict and evaluate on the test set
y_pred_w2v = svm_w2v_best.predict(X_test_w2v_scaled)
print(f"Word2Vec SVM Test Accuracy: {accuracy_score(y_test_w2v, y_pred_w2v):.4f}")
print(f"Word2Vec SVM Test Precision: {precision_score(y_test_w2v, y_pred_w2v):.4f}")
print(f"Word2Vec SVM Test Recall: {recall_score(y_test_w2v, y_pred_w2v):.4f}")
print(f"Word2Vec SVM Test F1 Score: {f1_score(y_test_w2v, y_pred_w2v):.4f}")
print(f"Word2Vec SVM Test Confusion Matrix:\n{confusion_matrix(y_test_w2v, y_pred_w2v)}")


[I 2024-10-17 20:34:32,413] A new study created in memory with name: no-name-f9e4b653-40f3-40e2-8d8d-0c871187cf5f
C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-10-17 20:35:31,086] Trial 0 finished with value: 0.992039368939065 and parameters: {'C': 3.065688351901044, 'kernel': 'poly', 'degree': 2}. Best is trial 0 with value: 0.992039368939065.
C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-10-17 20:36:25,713] Trial 1 finished with value: 0.9886085075702956 and parameters: {'C': 0.447839794252027, 'kernel': 'linear'}. Best is trial 0 with value: 0.992039368939065.
C:\Users\vatsa\AppData\Roaming

Best parameters for TF-IDF: {'C': 2.177271758954969, 'kernel': 'poly', 'degree': 2}
Best F1 Score for TF-IDF: 0.9926140477914555


C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-10-17 20:53:23,252] Trial 0 finished with value: 0.9904513888888888 and parameters: {'C': 1.5625324529194733, 'kernel': 'rbf'}. Best is trial 0 with value: 0.9904513888888888.
C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-10-17 20:53:43,136] Trial 1 finished with value: 0.8503600960256068 and parameters: {'C': 8.58309497772601, 'kernel': 'linear'}. Best is trial 0 with value: 0.9904513888888888.
C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consi

Best parameters for Word2Vec: {'C': 4.756772140965641, 'kernel': 'rbf'}
Best F1 Score for Word2Vec: 0.9926161864774866


C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


TF-IDF SVM Test Accuracy: 0.9903
TF-IDF SVM Test Precision: 0.9893
TF-IDF SVM Test Recall: 0.9919
TF-IDF SVM Test F1 Score: 0.9906
TF-IDF SVM Test Confusion Matrix:
[[3250   37]
 [  28 3418]]


C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Word2Vec SVM Test Accuracy: 0.9926
Word2Vec SVM Test Precision: 0.9902
Word2Vec SVM Test Recall: 0.9954
Word2Vec SVM Test F1 Score: 0.9928
Word2Vec SVM Test Confusion Matrix:
[[3253   34]
 [  16 3430]]


In [28]:
import optuna
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold

# Assuming you have already done the train-test split and vectorization:
# X_train_tfidf, X_test_tfidf: TF-IDF vectorized train and test sets (sparse)
# X_train_w2v, X_test_w2v: Word2Vec vectorized train and test sets
# y_train, y_test: Corresponding labels

# Split the training set into train/validation set
X_train_tfidf, X_val_tfidf, y_train_tfidf, y_val_tfidf = train_test_split(
    X_train_tfidf, y_train_tfidf, test_size=0.2, random_state=42, stratify=y_train_tfidf)

X_train_w2v, X_val_w2v, y_train_w2v, y_val_w2v = train_test_split(
    X_train_w2v, y_train_w2v, test_size=0.2, random_state=42, stratify=y_train_w2v)

# Cross-validation settings
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### Objective function for Optuna ###
def objective(trial, X_train, y_train):
    # Suggest hyperparameters for SVM
    C = trial.suggest_float('C', 0.1, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 4)
    else:
        degree = 3  # Default value for other kernels

    # Feature scaling for Word2Vec (dense matrix)
    if isinstance(X_train, np.ndarray):  
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)

    # Create SVM model
    svm = SVC(C=C, kernel=kernel, degree=degree, random_state=42, cache_size=10240, max_iter=2000)

    # Cross-validation scoring
    scores = cross_val_score(svm, X_train, y_train, cv=cv, scoring='f1')

    # Return mean F1 score
    return np.mean(scores)

### Step 1: Optimize for TF-IDF ###
def optimize_tfidf():
    study_tfidf = optuna.create_study(direction='maximize')
    study_tfidf.optimize(lambda trial: objective(trial, X_train_tfidf, y_train_tfidf), n_trials=20)

    # Best hyperparameters from Optuna
    print("Best parameters for TF-IDF:", study_tfidf.best_params)
    print("Best F1 Score for TF-IDF:", study_tfidf.best_value)

    return study_tfidf.best_params

### Step 2: Optimize for Word2Vec ###
def optimize_w2v():
    study_w2v = optuna.create_study(direction='maximize')
    study_w2v.optimize(lambda trial: objective(trial, X_train_w2v, y_train_w2v), n_trials=20)

    # Best hyperparameters from Optuna
    print("Best parameters for Word2Vec:", study_w2v.best_params)
    print("Best F1 Score for Word2Vec:", study_w2v.best_value)

    return study_w2v.best_params

### Step 3: Run the optimization and evaluate ###
best_params_tfidf = optimize_tfidf()  # Optimize for TF-IDF
best_params_w2v = optimize_w2v()  # Optimize for Word2Vec

### Step 4: Train and evaluate the best model for TF-IDF ###

svm_tfidf_best = SVC(
    C=best_params_tfidf['C'],
    kernel=best_params_tfidf['kernel'],
    degree=best_params_tfidf.get('degree', 3),
    random_state=42,
    cache_size=10240,
    max_iter=1000
)

# Fit the model on the train set and evaluate on the validation set
svm_tfidf_best.fit(X_train_tfidf, y_train_tfidf)

# Predict and evaluate on the validation set
y_pred_tfidf = svm_tfidf_best.predict(X_val_tfidf)
print(f"TF-IDF SVM Validation Accuracy: {accuracy_score(y_val_tfidf, y_pred_tfidf):.4f}")
print(f"TF-IDF SVM Validation Precision: {precision_score(y_val_tfidf, y_pred_tfidf):.4f}")
print(f"TF-IDF SVM Validation Recall: {recall_score(y_val_tfidf, y_pred_tfidf):.4f}")
print(f"TF-IDF SVM Validation F1 Score: {f1_score(y_val_tfidf, y_pred_tfidf):.4f}")
print(f"TF-IDF SVM Validation Confusion Matrix:\n{confusion_matrix(y_val_tfidf, y_pred_tfidf)}")

### Step 5: Train and evaluate the best model for Word2Vec ###

svm_w2v_best = SVC(
    C=best_params_w2v['C'],
    kernel=best_params_w2v['kernel'],
    degree=best_params_w2v.get('degree', 3),
    random_state=42,
    cache_size=10240,
    max_iter=1000
)

# Standardize the Word2Vec data before fitting
scaler = StandardScaler()
X_train_w2v_scaled = scaler.fit_transform(X_train_w2v)
X_val_w2v_scaled = scaler.transform(X_val_w2v)

# Fit the model
svm_w2v_best.fit(X_train_w2v_scaled, y_train_w2v)

# Predict and evaluate on the validation set
y_pred_w2v = svm_w2v_best.predict(X_val_w2v_scaled)
print(f"Word2Vec SVM Validation Accuracy: {accuracy_score(y_val_w2v, y_pred_w2v):.4f}")
print(f"Word2Vec SVM Validation Precision: {precision_score(y_val_w2v, y_pred_w2v):.4f}")
print(f"Word2Vec SVM Validation Recall: {recall_score(y_val_w2v, y_pred_w2v):.4f}")
print(f"Word2Vec SVM Validation F1 Score: {f1_score(y_val_w2v, y_pred_w2v):.4f}")
print(f"Word2Vec SVM Validation Confusion Matrix:\n{confusion_matrix(y_val_w2v, y_pred_w2v)}")


[I 2024-10-23 20:09:29,425] A new study created in memory with name: no-name-a7bab8d2-6853-4d5c-b859-abe1440c7acf
C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processin

Best parameters for TF-IDF: {'C': 6.179525546807796, 'kernel': 'rbf'}
Best F1 Score for TF-IDF: 0.9903534998756897


C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\vatsa\AppData\Roaming\Python\Python312\

Best parameters for Word2Vec: {'C': 5.981838190834487, 'kernel': 'rbf'}
Best F1 Score for Word2Vec: 0.9909293275440831


C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


TF-IDF SVM Validation Accuracy: 0.9918
TF-IDF SVM Validation Precision: 0.9873
TF-IDF SVM Validation Recall: 0.9967
TF-IDF SVM Validation F1 Score: 0.9920
TF-IDF SVM Validation Confusion Matrix:
[[2617   35]
 [   9 2726]]


C:\Users\vatsa\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Word2Vec SVM Validation Accuracy: 0.9913
Word2Vec SVM Validation Precision: 0.9863
Word2Vec SVM Validation Recall: 0.9967
Word2Vec SVM Validation F1 Score: 0.9915
Word2Vec SVM Validation Confusion Matrix:
[[2614   38]
 [   9 2726]]


<h1> 2. Random Forest </h2>

<h1> OPTUNA CODE </h2>

In [52]:
import optuna
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Assuming you have already done the train-test split and vectorization:
# X_train_tfidf, X_test_tfidf: TF-IDF vectorized train and test sets
# X_train_w2v, X_test_w2v: Word2Vec vectorized train and test sets
# y_train, y_test: Corresponding labels

### Objective function for Optuna ###
def objective(trial, X_train, y_train, X_test, y_test):
    # Suggest hyperparameters to optimize
    n_estimators = trial.suggest_int('n_estimators', 100,300, step=10)
    max_depth = trial.suggest_categorical('max_depth', [None, 10, 20, 30, 40, 50])
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    
    # Define the model with the suggested hyperparameters
    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=42,
        n_jobs=-1
    )
    
    # Train the model
    rf.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = rf.predict(X_test)
    
    # Calculate the F1 score (can also be other metrics if needed)
    f1 = f1_score(y_test, y_pred)
    
    return f1  # Optuna will maximize this metric

### Step 1: Optimize for TF-IDF ###
def optimize_tfidf():
    study_tfidf = optuna.create_study(direction='maximize')
    study_tfidf.optimize(lambda trial: objective(trial, X_train_tfidf, y_train_tfidf, X_test_tfidf, y_test_tfidf), n_trials=50)
    
    # Best hyperparameters from Optuna
    print("Best parameters for TF-IDF:", study_tfidf.best_params)
    print("Best F1 Score for TF-IDF:", study_tfidf.best_value)

    return study_tfidf.best_params

### Step 2: Optimize for Word2Vec ###
def optimize_w2v():
    study_w2v = optuna.create_study(direction='maximize')
    study_w2v.optimize(lambda trial: objective(trial, X_train_w2v, y_train_w2v, X_test_w2v, y_test_w2v), n_trials=50)
    
    # Best hyperparameters from Optuna
    print("Best parameters for Word2Vec:", study_w2v.best_params)
    print("Best F1 Score for Word2Vec:", study_w2v.best_value)

    return study_w2v.best_params

### Step 3: Run the optimization and evaluate ###
best_params_tfidf = optimize_tfidf()  # Optimize for TF-IDF
best_params_w2v = optimize_w2v()  # Optimize for Word2Vec

### Step 4: Train and evaluate the best model for TF-IDF ###
rf_tfidf_best = RandomForestClassifier(
    n_estimators=best_params_tfidf['n_estimators'],
    max_depth=best_params_tfidf['max_depth'],
    min_samples_split=best_params_tfidf['min_samples_split'],
    random_state=42,
    n_jobs=-1
)

# Fit the model
rf_tfidf_best.fit(X_train_tfidf, y_train_tfidf)

# Predict and evaluate on the test set
y_pred_tfidf = rf_tfidf_best.predict(X_test_tfidf)
print(f"TF-IDF Random Forest Test Accuracy: {accuracy_score(y_test_tfidf, y_pred_tfidf):.4f}")
print(f"TF-IDF Random Forest Test Precision: {precision_score(y_test_tfidf, y_pred_tfidf):.4f}")
print(f"TF-IDF Random Forest Test Recall: {recall_score(y_test_tfidf, y_pred_tfidf):.4f}")
print(f"TF-IDF Random Forest Test F1 Score: {f1_score(y_test_tfidf, y_pred_tfidf):.4f}")
print(f"TF-IDF Random Forest Test Confusion Matrix:\n{confusion_matrix(y_test_tfidf, y_pred_tfidf)}")

### Step 5: Train and evaluate the best model for Word2Vec ###
rf_w2v_best = RandomForestClassifier(
    n_estimators=best_params_w2v['n_estimators'],
    max_depth=best_params_w2v['max_depth'],
    min_samples_split=best_params_w2v['min_samples_split'],
    random_state=42,
    n_jobs=-1
)

# Fit the model
rf_w2v_best.fit(X_train_w2v, y_train_w2v)

# Predict and evaluate on the test set
y_pred_w2v = rf_w2v_best.predict(X_test_w2v)
print(f"Word2Vec Random Forest Test Accuracy: {accuracy_score(y_test_w2v, y_pred_w2v):.4f}")
print(f"Word2Vec Random Forest Test Precision: {precision_score(y_test_w2v, y_pred_w2v):.4f}")
print(f"Word2Vec Random Forest Test Recall: {recall_score(y_test_w2v, y_pred_w2v):.4f}")
print(f"Word2Vec Random Forest Test F1 Score: {f1_score(y_test_w2v, y_pred_w2v):.4f}")
print(f"Word2Vec Random Forest Test Confusion Matrix:\n{confusion_matrix(y_test_w2v, y_pred_w2v)}")


[I 2024-10-17 14:59:14,855] A new study created in memory with name: no-name-185cf3b7-bbd5-4814-90f9-b8d79c84f0ba
[I 2024-10-17 14:59:17,634] Trial 0 finished with value: 0.9684805653710248 and parameters: {'n_estimators': 180, 'max_depth': 30, 'min_samples_split': 9}. Best is trial 0 with value: 0.9684805653710248.
[I 2024-10-17 14:59:22,726] Trial 1 finished with value: 0.9748042704626334 and parameters: {'n_estimators': 280, 'max_depth': 40, 'min_samples_split': 10}. Best is trial 1 with value: 0.9748042704626334.
[I 2024-10-17 14:59:24,800] Trial 2 finished with value: 0.9683347469607012 and parameters: {'n_estimators': 140, 'max_depth': 30, 'min_samples_split': 6}. Best is trial 1 with value: 0.9748042704626334.
[I 2024-10-17 14:59:28,279] Trial 3 finished with value: 0.975505553973227 and parameters: {'n_estimators': 150, 'max_depth': 40, 'min_samples_split': 7}. Best is trial 3 with value: 0.975505553973227.
[I 2024-10-17 14:59:36,352] Trial 4 finished with value: 0.978978978978

Best parameters for TF-IDF: {'n_estimators': 270, 'max_depth': None, 'min_samples_split': 4}
Best F1 Score for TF-IDF: 0.9874077290490665


[I 2024-10-17 15:05:24,904] Trial 0 finished with value: 0.9862458375561025 and parameters: {'n_estimators': 200, 'max_depth': 10, 'min_samples_split': 9}. Best is trial 0 with value: 0.9862458375561025.
[I 2024-10-17 15:05:43,229] Trial 1 finished with value: 0.987385819921705 and parameters: {'n_estimators': 190, 'max_depth': 20, 'min_samples_split': 7}. Best is trial 1 with value: 0.987385819921705.
[I 2024-10-17 15:06:06,291] Trial 2 finished with value: 0.9879587987813724 and parameters: {'n_estimators': 270, 'max_depth': 50, 'min_samples_split': 2}. Best is trial 2 with value: 0.9879587987813724.
[I 2024-10-17 15:06:15,871] Trial 3 finished with value: 0.9873748367435786 and parameters: {'n_estimators': 120, 'max_depth': 50, 'min_samples_split': 4}. Best is trial 2 with value: 0.9879587987813724.
[I 2024-10-17 15:06:34,657] Trial 4 finished with value: 0.9879657822241554 and parameters: {'n_estimators': 230, 'max_depth': 50, 'min_samples_split': 9}. Best is trial 4 with value: 0.

Best parameters for Word2Vec: {'n_estimators': 130, 'max_depth': 40, 'min_samples_split': 2}
Best F1 Score for Word2Vec: 0.9882455376578145
TF-IDF Random Forest Test Accuracy: 0.9871
TF-IDF Random Forest Test Precision: 0.9805
TF-IDF Random Forest Test Recall: 0.9945
TF-IDF Random Forest Test F1 Score: 0.9874
TF-IDF Random Forest Test Confusion Matrix:
[[3246   68]
 [  19 3411]]
Word2Vec Random Forest Test Accuracy: 0.9880
Word2Vec Random Forest Test Precision: 0.9838
Word2Vec Random Forest Test Recall: 0.9927
Word2Vec Random Forest Test F1 Score: 0.9882
Word2Vec Random Forest Test Confusion Matrix:
[[3258   56]
 [  25 3405]]
